In [35]:
import numpy as np

#import pandas as pd

import matplotlib.pyplot as plt

from aide_design import physchem as pc

from aide_design import expert_inputs as exp

from aide_design import utility as ut

from aide_design.units import unit_registry as u

from aide_design.unit_process_design.prefab import lfom_prefab_functional as lfom

In [36]:
## Set parameters of plant below

In [37]:
Q_plant=1*u.L/u.s
T = u.Quantity(20,u.degC)
NuPaCl = pc.viscosity_kinematic(T)
Headloss=10*u.cm
StockCl=70*u.gram/u.L
DesiredCl_Concentration=3*u.mg/u.L
RatioError = 0.1
KMinor = 2

In [38]:
Q_CDC=(Q_plant*DesiredCl_Concentration/StockCl).to(u.mL/u.s)
print(Q_CDC) ##mL/sec
print(Q_CDC.to(u.L/u.hour)) ##L/hr

0.04285714285714286 milliliter / second
0.1542857142857143 liter / hour


In [39]:
##Find diameter of dooosing toob

In [40]:
DiamTubeArray = np.array(np.arange(1/16,3/16,1/16)) * u.inch ## 1/16 or 1/8 inch tubing
print(DiamTubeArray) 

VelTubeMax = (((RatioError * 2 * Headloss * pc.gravity) / KMinor)**(1/2)).to(u.meter/u.s)
print('The maximum average velocity in a dosing tube is', ut.sig(VelTubeMax,3))

[ 0.0625  0.125 ] inch
The maximum average velocity in a dosing tube is 0.313 m/s


In [41]:
##Flow rates for tubes

In [42]:
def flow_cdc_max(diam, VelTubeMax): 
    Flow = pc.area_circle(diam) * (VelTubeMax)
    return Flow
        
FlowMaxArray = flow_cdc_max(DiamTubeArray, VelTubeMax).to(u.mL/u.s)
print(FlowMaxArray)

[ 0.61983727  2.47934908] milliliter / second


In [43]:
##Lengths for each tube

In [47]:
def length_tube(flow_max, diam, headloss_max, nu, k_minor):
    "Returns the length of tube necessary to handle the maximum flow."
    L = (((pc.gravity * headloss_max * np.pi * diam**4)/
                           (128 * nu * flow_max))-
                    ((k_minor * flow_max)/
                             (16 * np.pi * nu)))
    return L 
length_dosingtube=length_tube(FlowMaxArray, DiamTubeArray,Headloss, NuPaCl,KMinor)
print(length_dosingtube.to(u.m))

[ 0.22117655  0.88470622] meter


In [ ]:
##Both are reasonably lengthed. .22 would be easier to wrap around so I conclude that we should use the 1/16th diameter tubing. 
